Python optimization 

replace with precalculations 
-> given a recurring call to the same expression, python 
instead of re-calculating it, replaces the redundant expression with 
the original value 

a= 64*40= 2460 #---stores 2460 at 0xFFDD
b= 64*40---> replaces with call at 0xFFDD as above instead of recalculation


same applies to short sequences <20
so if 
(1,2) * 5 ->(1,2,1,2,1,2,1,2,1,2)
'abc'*3-> abcabcabc
'hello' + 'world'--> stored
'the quick brown fox jumps' *3 -> not precalculated as >20

 

Membership tests-> 
mutables types are replaced by immutable 

for membership test such as -> 
    if e in [1,2,3]:
    evaluate to constant expression (which it does in this case as all of 1,2,3 are 
    constants, would've not been the case if that wasn't true) then python replaces 
    that by 
    
    if e in (1,2,3) -> tuple counter part 
    
    similar conversions, 
    > list-> tuples 
    > sets-> frozensets 
    
    Set membership is much faster than list or tuple membership 
    
    so mostly, unless dealing with variables we should instead write 
    if e in {1,2,3}: # using sets 

In [2]:
def my_func():
    a= 24*60
    b= (1,2)* 5
    c=  'abc'* 3
    d= 'ab' *11
    e= 'the quick brown' *5
    f= ['a', 'b']*3 

In [3]:
my_func.__code__.co_consts

(None,
 1440,
 (1, 2, 1, 2, 1, 2, 1, 2, 1, 2),
 'abcabcabc',
 'ababababababababababab',
 'the quick brownthe quick brownthe quick brownthe quick brownthe quick brown',
 'a',
 'b',
 3)

in the above snippet which was fetched from code constants function in 
the code module, we see that python has cleverly already calculated 
the expressions such as 1440 et cetera.

similar to this, we have 

    def my_func(e):
        if e in [1,2,3]:
            pass

the above code will be replaced by a static lookup that can be easily reused

In [4]:
def my_func(e):
    if e in [1,2,3]:
        pass

In [5]:
my_func.__code__.co_consts

(None, (1, 2, 3))

we see python actually stores the list as a tuple correspendence 
as doing so allows to immutability safety. 

using sets is much more efficient than using a list or tuple because 
both list and tuple need to traverse through the entire object in order 
to process an instruction however, 

a hashmap based structure such as set can perform this several times faster.

In [8]:
import time, string
def membership_test(n, container):
    for i in range(n):
        if 'z' in container:
            pass

In [9]:
string.ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [10]:
char_list= list(string.ascii_letters)
char_tuple= tuple(string.ascii_letters)
char_set= set(string.ascii_letters)

In [13]:
print(char_list,"\t\n", char_list, "\t\n", char_tuple)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'] 	
 ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'] 	
 ('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z')


In [15]:
start= time.perf_counter()
membership_test(100000000,char_list)
end= time.perf_counter()
print(" it takes", start-end)

 it takes -40.41606290000027


In [16]:
start= time.perf_counter()
membership_test(100000000,char_tuple)
end= time.perf_counter()
print(" it takes", start-end)

 it takes -41.72024049999982


so they both take roughly the same amount of time for traversing 10mil elements

In [17]:
start= time.perf_counter()
membership_test(100000000,char_set)
end= time.perf_counter()
print(" it takes", start-end)

 it takes -4.587058999999954


we see that a set takes a blazingly fast approach and returns at 4 seconds